# Generate recipes from chat GPT

In [1]:
import pandas as pd
import numpy as np 
from dotenv import load_dotenv
import os
import openai
import json 

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("API_KEY")

In [4]:
openai.api_key = api_key

In [5]:
models = openai.Model.list()

In [6]:
models

<OpenAIObject list at 0x7fec0c752e00> JSON: {
  "object": "list",
  "data": [
    {
      "id": "text-search-babbage-doc-001",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev"
    },
    {
      "id": "gpt-4",
      "object": "model",
      "created": 1687882411,
      "owned_by": "openai"
    },
    {
      "id": "gpt-3.5-turbo-16k",
      "object": "model",
      "created": 1683758102,
      "owned_by": "openai-internal"
    },
    {
      "id": "curie-search-query",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev"
    },
    {
      "id": "text-davinci-003",
      "object": "model",
      "created": 1669599635,
      "owned_by": "openai-internal"
    },
    {
      "id": "text-search-babbage-query-001",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev"
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai"
    },
   

In [7]:
query_ingredients = """Please give me the ingredients and preparation steps, 
separated by ';' for the following recipe {recipe_name}"""

In [8]:
user_text = """
                Please give me {number}, {temporal} nutrition plan. 
                For each nutrition plan give me {n_meals} meals in this order:
                {breakfast}, {snack}, {lunch}, {snack}, {dinner} that in total sum up 
                maximum {maximum_calories}kcals per {daily_calorie_restriction}. 
                {cultural_restrictions}
            """

In [9]:
dict_parameters = {"number": "7",
                   "temporal": "daily",
                   "n_meals": "5",
                   "breakfast": "breakfast",
                   "snack": "snacks",
                   "lunch": "lunch",
                   "dinner": "dinner",
                   "maximum_calories": 1000,
                   "daily_calorie_restriction": "day",
                   "cultural_restrictions": ""
                   }

In [10]:
def generate_nutrition_plans(user_text: str, samples_to_generated: int = 10, model: str = "gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=model,
        n = samples_to_generated,
        messages =[
            {"role": "system", "content": "Generating recipes"},
            {"role": "user", "content": user_text}
            ]
        )
    return response


In [58]:
from typing import List
def generate_ingredients_for_recipes(query_template: str, recipes:List):
    answer = {}
    for recipe in recipes:
        try:
            print(f"Processing recipe: {recipe}")
            current_recipe = recipe.strip()
            processed_query = query_template.format(recipe_name=current_recipe)
            response = generate_nutrition_plans(processed_query, 
                                                samples_to_generated = 1, 
                                                model = "gpt-3.5-turbo-16k")
            dict_text = dict(response["choices"][0])
            answer[recipe] = dict_text["message"]["content"]
        except Exception as e:
            print(f"Exception: {e} for recipe: {recipe}")
            answer[recipe] = ""
            continue
    return answer 

In [86]:
# load recipes file 
recipes_ds = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/recipes/processed_recipes_dataset_id.csv", 
                         index_col=0,
                         sep="|")

In [88]:
# check bad processing recipes 
negative_mask = recipes_ds["calories"] < 0

In [90]:
second_check_recipes = recipes_ds[negative_mask]

In [125]:
# extract calories 
import re
def extract_calories(raw_text: str):
    # normalize text 
    temp_text = raw_text.lower()
    pattern = r'calories\s*per\s*portion[:]*\s*(\d+)\s*kcal'
    matches = re.findall(pattern, temp_text)
    if matches:
        numbers = [int(match) for match in matches]
        return numbers[0]
    else:
        print("Not found string")
        return None

In [126]:
test = second_check_recipes["title"].tolist()

In [132]:
unique_recipes_2 = set(test).difference(set(unique_recipes))

In [134]:
unique_recipes_2 = list(unique_recipes_2)

In [92]:
raw_text_list = second_check_recipes["raw_text"].tolist()

In [99]:
print(raw_text_list[0])

2. Overnight Oats: Calories per portion: 300kcal, Total portions: 1, Allergic warnings: None, Origin: N/A



In [120]:
import re
pattern = r'calories\s*per\s*portion[:]*\s*(\d+)\s*kcal'

In [121]:
raw_text_list[0].lower()

'2. overnight oats: calories per portion: 300kcal, total portions: 1, allergic warnings: none, origin: n/a\n'

In [122]:
matches = re.findall(pattern, raw_text_list[0].lower())

In [123]:
if matches:
    numbers = [int(match) for match in matches]
else:
    print("Not found string")

In [124]:
numbers

[300]

In [84]:
mask = recipes_ds["calories"] > 0
recipes_ds = recipes_ds.loc[mask, :]
unique_recipes = np.unique(recipes_ds["title"])
print(f"Number of unique recipes: {len(unique_recipes)}")

Number of unique recipes: 5028


In [61]:
mask = [True if r in unique_recipes else False for r in recipes_ds["title"].tolist()]
unique_ds = recipes_ds.loc[mask, :]
print(unique_ds.shape)
print(unique_ds.head(4))

(5028, 7)
                     title                                           raw_text  \
0              Fruit Salad  1. Fruit Salad: 70 calories per portion, 4 por...   
1           Vegan Pancakes  2. Vegan Pancakes: 150 calories per portion, 4...   
2   Overnight Chia Pudding  3. Overnight Chia Pudding: 200 calories per po...   
3            Avocado Toast  4. Avocado Toast: 250 calories per portion, 2 ...   

   meal_type cultural_restriction  calories                allergies recipeId  
0  breakfast                vegan      70.0     contains fruits only   food_0  
1  breakfast                vegan     150.0                      NaN   food_1  
2  breakfast                vegan     200.0  contains nuts (almonds)   food_2  
3  breakfast                vegan     250.0                      NaN   food_3  


In [62]:
recipes_list = unique_ds["title"].tolist()

In [144]:
len(recipes_list)

5028

In [154]:
total_results = []
chunk_size = 50
for i in range(3500, 4000, chunk_size):
    print(f"processing chunk {i}")
    ingredients_dict = generate_ingredients_for_recipes(query_ingredients,
                                                    recipes=recipes_list[i:i+chunk_size])
    total_results.append(ingredients_dict)

processing chunk 3500
Processing recipe:  Mexican Beef Tacos
Processing recipe:  Greek Souvlaki Wrap
Processing recipe:  Chicken and Vegetable Noodle Stir Fry
Processing recipe:  Turkish Pide with Spinach and Feta
Processing recipe:  Spanish Gazpacho
Processing recipe:  Chicken Shawarma Bowl
Processing recipe:  Italian Margherita Pizza
Processing recipe:  Moroccan Lamb Tagine
Processing recipe:  Greek Tabbouleh Salad
Processing recipe:  Chicken and Vegetable Fajitas
Processing recipe:  Indonesian Nasi Goreng
Processing recipe:  Greek Moussaka Wrap
Processing recipe:  Lebanese Chicken Fatteh
Processing recipe:  Mexican Chicken Quesadillas
Processing recipe:  Greek Feta and Spinach Stuffed Mushrooms
Processing recipe:  Moroccan Chicken Pastilla
Processing recipe:  Chinese Hot and Sour Soup
Processing recipe:  Chicken Katsu Curry
Processing recipe:  Greek Lamb Gyros
Processing recipe:  Vietnamese Pho
Processing recipe:  Turkish Lahmacun
Processing recipe:  Moroccan Chicken and Vegetable S

In [155]:
final_dict = {}
for item in total_results:
    final_dict.update(item)

In [156]:
with open("extended_recipes_raw_3500_4000.json", 'w') as fp:
    json.dump(final_dict, fp)

In [29]:
data_dict = dict(ingredients_dict[' Fruit Salad']["choices"][0])

In [33]:
data_dict["message"]["content"]

'Ingredients:\n- 2 cups mixed fresh fruits (such as strawberries, blueberries, grapes, and melons)\n- 1 cup sliced banana\n- 1 cup sliced pineapple\n- 1 cup diced apples\n- 1 cup diced oranges\n- 1 tablespoon honey\n- Fresh mint leaves for garnish (optional)\n\nPreparation:\n1. Wash and cut all the fruits into bite-sized pieces.\n2. In a mixing bowl, combine the mixed fresh fruits, sliced banana, sliced pineapple, diced apples, and diced oranges.\n3. Drizzle the honey over the fruit mixture and gently toss to coat all the fruits with the honey.\n4. Let the fruit salad sit in the refrigerator for at least 30 minutes to allow the flavors to meld together.\n5. Serve chilled and garnish with fresh mint leaves if desired.\n6. Enjoy your refreshing fruit salad!'

In [122]:
def generate_meals():
    food_restrictions = ["kosher", "None"]
    user_text = """Please give me {number} different food recipes for {type_food} between 
    {low_calories}kcals and {high_calories}kcals per portion. For each recipe, 
    give me the total amount of calories per portion, total portions, allergic warnings, 
    and origin country if available.
    {cultural_restrictions}
    """
    dict_parameters = {"number": 50,
                       "type_food": "breakfast",
                       "low_calories": 20,
                       "high_calories": 1000,
                       "cultural_restrictions": ""
                       }
    type_food_max_calories = {"breakfast":900}
                            #   "morning snacks": 300,
                            #   "afternoon snacks": 300,
                            #   "lunch": 1200,
                            #   "dinner": 600}
    for type_food, max_calories in type_food_max_calories.items():
        for cultural_restriction in food_restrictions:
            print(f"Generating recipes for {type_food} with max calories: {max_calories} with cultural restrictions: {cultural_restriction}")
            if cultural_restriction != "None":
                dict_parameters["cultural_restrictions"] = f" The recipes should be {cultural_restriction}."
                dict_parameters["number"] = 100
            else:
                dict_parameters["cultural_restrictions"] = ""
                dict_parameters["number"] = 200
            dict_parameters["type_food"] = type_food
            dict_parameters["max_calories"] = max_calories
            user_text_formate = user_text.format(**dict_parameters)
            print(f"Text to sent to chat: {user_text_formate}")
            response = generate_nutrition_plans(user_text_formate, 
                                                samples_to_generated = 10, 
                                                model = "gpt-3.5-turbo")
            with open(f"{type_food}_{cultural_restriction}.json", "w", encoding='utf-8') as fp:
                json.dump(response, fp, ensure_ascii=False, indent=4)
            print("Finished!!!")
            print("."*300)
    print("success!!!")

In [123]:
generate_meals()

Generating recipes for breakfast with max calories: 900 with cultural restrictions: kosher
Text to sent to chat: Please give me 100 different food recipes for breakfast between 
    20kcals and 1000kcals per portion. For each recipe, 
    give me the total amount of calories per portion, total portions, allergic warnings, 
    and origin country if available.
     The recipes should be kosher.
    
Finished!!!
............................................................................................................................................................................................................................................................................................................
Generating recipes for breakfast with max calories: 900 with cultural restrictions: None
Text to sent to chat: Please give me 200 different food recipes for breakfast between 
    20kcals and 1000kcals per portion. For each recipe, 
    give me the total amount of calories per portion

In [63]:
generate_nutrition_plans()

Generating recipes with restriction: vegan per calories: 2000
Generated restriction
Generating recipes with restriction: vegetarian per calories: 2000
Generated restriction
Generating recipes with restriction: halal per calories: 2000
Generated restriction
Generating recipes with restriction: kosher per calories: 2000
Generated restriction
Generating recipes with restriction: None per calories: 2000
Generating recipes with restriction: vegan per calories: 2500
Generated restriction
Generating recipes with restriction: vegetarian per calories: 2500
Generated restriction
Generating recipes with restriction: halal per calories: 2500
Generated restriction
Generating recipes with restriction: kosher per calories: 2500
Generated restriction
Generating recipes with restriction: None per calories: 2500
Generating recipes with restriction: vegan per calories: 3000
Generated restriction
Generating recipes with restriction: vegetarian per calories: 3000
Generated restriction
Generating recipes wi

In [36]:
food_restrictions = ["vegan_observant", "vegetarian_observant", "halal_observant", "kosher_observant", "None"]
food_restriction_probs = [0.2, 0.3, 0.05, 0.05, 0.4]

In [51]:
user_text = """
Please give me {number}, {temporal} nutrition plan. 
For each nutrition plan give me {n_meals} meals in this order:
{breakfast}, {snack}, {lunch}, {snack}, {dinner} that in total sum up 
maximum {maximum_calories}kcals per {daily_calorie_restriction}. 
{cultural_restrictions}
"""

In [54]:
dict_parameters = {"number": "7",
                   "temporal": "daily",
                   "n_meals": "5",
                   "breakfast": "breakfast",
                   "snack": "snack",
                   "lunch": "lunch",
                   "dinner": "dinner",
                   "maximum_calories": 1000,
                   "daily_calorie_restriction": "day",
                   "cultural_restrictions": ""
                   }

In [55]:
user_text.format(**dict_parameters)

'\nPlease give me 7, daily nutrition plan. \nFor each nutrition plan give me 5 meals in this order:\nbreakfast, snack, lunch, snack, dinner that in total sum up \nmaximum 1000kcals per day .\n'

In [40]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    n = 10,
    messages =[
        {"role": "system", "content": "Generating recipes"},
        {"role": "user", "content": user_text.format(dict_parameters)}
    ]
)

In [41]:
# save response 
with open("without_restrictions.json", 'w', encoding='utf-8') as f:
    json.dump(response, f, ensure_ascii=False, indent=4)

In [34]:
response

<OpenAIObject chat.completion id=chatcmpl-7gwpYYexdutRj82VWQDqQ69B44HBJ at 0x7facbc7fe310> JSON: {
  "id": "chatcmpl-7gwpYYexdutRj82VWQDqQ69B44HBJ",
  "object": "chat.completion",
  "created": 1690470276,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Sure! Here's a 10-day nutrition plan with five meals per day, each totaling maximum 1000 calories:\n\nDay 1:\nBreakfast: Veggie Omelette (300 calories)\n- 2 eggs (140 calories)\n- \u00bc cup diced bell peppers (10 calories)\n- \u00bc cup diced onions (10 calories)\n- \u00bc cup spinach (3 calories)\n- 1 slice of whole wheat toast (70 calories)\nSnack: Apple slices with almond butter (150 calories)\n- 1 medium apple (95 calories)\n- 1 tablespoon almond butter (55 calories)\nLunch: Greek Salad (300 calories)\n- 2 cups mixed greens (20 calories)\n- \u00bd cup cherry tomatoes (13 calories)\n- \u00bc cup sliced cucumbers (4 calories)\n- \u00bc cup crum

In [32]:
recipes = response["choices"][0]["message"]["content"]

In [35]:
with open("general_raw_recipes_10_days_1000.txt", "w") as fp:
    fp.writelines(recipes)

In [22]:
print(recipes)

Recipe 1: Breakfast - Scrambled Egg Toast
- 2 slices of whole wheat bread (160 calories)
- 2 large eggs (140 calories)
- 1/4 cup of chopped vegetables (20 calories)
- Salt and pepper to taste
- Non-stick cooking spray

Instructions:
1. Lightly coat a non-stick skillet with cooking spray and heat over medium heat.
2. In a bowl, whisk the eggs together and season with salt and pepper.
3. Add the chopped vegetables to the skillet and sauté for a couple of minutes until slightly softened.
4. Pour the whisked eggs into the skillet and scramble them with the vegetables until fully cooked.
5. Toast the bread slices and serve the scrambled eggs on top. Enjoy!

Recipe 2: Snack - Greek Yogurt Parfait
- 1 cup of plain Greek yogurt (130 calories)
- 1/4 cup of granola (100 calories)
- 1/2 cup of mixed berries (70 calories)
- 1 teaspoon of honey (20 calories)

Instructions:
1. In a bowl or glass, start by layering half of the Greek yogurt.
2. Add half of the granola on top of the yogurt.
3. Layer ha